In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/used-cars-dataset-from-cardekhocom/Cardekho_Extract.csv")

Dataframe marks almost all columns as object.

In [ ]:
df.dtypes

In [ ]:
df_original = df.copy()

Converting the seats column from object to integer. With mode seat number (5) assigned to missing values.

In [ ]:
df['seats'] = df['seats'].str[5:]

In [ ]:
df[df['seats']=="N/A"]['full_name']

In [ ]:
df.at[16375, 'seats'] = 5

In [ ]:
dropping_df = df[df['seats'].isnull() & df['full_name'].isnull()]
df = pd.concat([df, dropping_df]).drop_duplicates(keep=False)

In [ ]:
df['seats'].astype(float)

In [ ]:
seat_mode= int(df['seats'].mode())
df['seats'].replace({np.nan: seat_mode}, inplace=True)

In [ ]:
df['seats'] = df['seats'].astype(int)

`df['seats']` is ready 🎉. Now converting `Max Power` column to appropriate data type and replacing missing values with mean values.

In [ ]:
import re

In [ ]:
def engine_power_extractor(text):
    try:
        return re.match(r"(Max Power)([0-9.]+)( bhp)", text).group(2)
    except Exception:
        return np.nan

In [ ]:
df['max_power'] = df['max_power'].apply(lambda x: engine_power_extractor(x))

In [ ]:
df['max_power'] = df['max_power'].astype(float)
mp_mean = df['max_power'].mean()
df['max_power'].replace(np.nan, mp_mean, inplace=True)

In [ ]:
df.describe()

In [ ]:
def extractor(text):
    try:
        return re.match(r"(Engine)([0-9]+)( CC)", text).group(2)
    except Exception:
        return np.nan

In [ ]:
df['engine'] = df['engine'].apply(lambda x: extractor(x))

In [ ]:
df['engine'] = df['engine'].astype(float)

In [ ]:
data_mean = df['engine'].mean()

In [ ]:
df['engine'].replace(np.nan, data_mean, inplace=True)

In [ ]:
var = []
for x in set(df['mileage']):
    var.append(x.split()[-1])
set(var)

In [ ]:
for x in set(var):
    print(x)
    print(sum(df['mileage'].str.contains(x)))

In [ ]:
df[df['mileage'].str.contains("km/kg")]['fuel_type'].value_counts()

In [ ]:
df['seller_type'].value_counts()

In [ ]:
km_extractor = lambda kms: "".join(list(filter(str.isdigit, kms)))

In [ ]:
df['km_driven'] = df['km_driven'].apply(km_extractor)
df['km_driven'] = df['km_driven'].astype(int, )

In [ ]:
df['year'] = df['year'].apply(int)
df['year'] = df['year'].astype(int)

In [ ]:
df.rename(columns={"new-price": "new_price"}, inplace=True)

In [ ]:
import random

In [ ]:
def indian_amount_conv(amount):
    if "Lakh" in amount[1]:
        return float(amount[0])  * 100_000
    elif "Cr" in amount[1]:
        return float(amount[0])  * 10_000_000

In [ ]:
good_data = []
for row in df['new_price']:
    if type(row) == float:
        good_data.append(row)
        continue
        
    price_data = row[29:]
    if "Cr-" in price_data:
        data_splitted = [price_data.split("-")[0].replace("Cr", "")] + price_data.split("-")[1].split()
        data_splitted[:2] = map(float, data_splitted[:2]) # [1.72, 1.84, 'Cr*']
        amount = [sum(data_splitted[:2]) /2 ] + [data_splitted[-1]] # [15.265, 'Lakh*']; Mean value of the range
        good_data.append(indian_amount_conv(amount)) 

    elif "-" in price_data:
        data_splitted = [price_data.split("-")[0]] + price_data.split("-")[1].split()
        data_splitted[:2] = map(float, data_splitted[:2])
        if data_splitted[0] > data_splitted[1]:
            data_splitted[0] = data_splitted[0]/100
        amount = [sum(data_splitted[:2]) /2 ] + [data_splitted[-1]]
        good_data.append(indian_amount_conv(amount))

    else:
        amount = price_data.split()
        good_data.append(indian_amount_conv(amount))

In [ ]:
df.head()

In [ ]:
for price in set(df['selling_price']):
    if not ("Lakh" in price or "Cr" in price):
        print(price)

In [ ]:
price_data = []
for price in df['selling_price']:
    if ("Lakh" in price or "Cr" in price):
        price = price.split()
        price_data.append(indian_amount_conv(price))
    else:
        float(price[:-1].replace(",", ""))
        price_data.append(price)

In [ ]:
df["selling_price_formatted"] = price_data

In [ ]:
df.head()

In [ ]:
df.to_csv("Formatted Indian Used Car.csv")